In [15]:
import pandas as pd
from Declare4Py.ProcessMiningTasks.ASPLogGeneration.ASPUtils.damerauLevenshtein import DamerauLevenshteinDistance
import os
from pprint import pprint

In [16]:
%%time

Cols_Name = ["Configuration", "Distance", "Time", "Frequency"]
threads_dict = {"4": [], "6": [], "8": [],"10": [],"12": []}

for file in os.listdir("Log_results"):
    
    stripped_file = file.split("_")
    
    time = stripped_file[1]
    config = stripped_file[3]
    thread = stripped_file[5]
    freq = stripped_file[7].split(".c")[0]
    distance = DamerauLevenshteinDistance.analize_csv(f"Log_results/{file}")
    
    threads_dict[thread].append([config, distance, time, freq])
    
    try:
        os.rename("Log_results/" + file, f"Log_results/similarity_{distance}_" + file)
    except FileNotFoundError:
        print("file not found")
    


CPU times: total: 11.4 s
Wall time: 11.5 s


In [17]:
for key, value in threads_dict.items():
    df = pd.DataFrame(value, columns=Cols_Name)
    df.to_csv(f"Log_results/similarity_report_threads_{key}.csv")

In [17]:
sorted_list = sorted(similarity_dict.items())
filtered_list = []

threshold = 45

for csv_file in sorted_list:
    if csv_file[1] > threshold:
        filtered_list.append(csv_file)
        
filtered_list.sort(key = lambda x: x[1], reverse = True)

files = [file[0] for file in filtered_list]

def get_similarity(csv_file: str):
    for name, similarity in filtered_list:
        if name == csv_file:
            return similarity
    return 0

for file in os.listdir("Log_results"):
    if file not in files:
        try:
            os.remove(f"Log_results/{file}")
        except FileNotFoundError:
            pass
    else:
        try:
            os.rename("Log_results/" + file, f"Log_results/similarity_{get_similarity(file)}_" + file)
        except FileNotFoundError:
            print("file not found")
            pass
            

In [1]:
sorted_list = sorted(similarity_dict.items())

def get_similarity(csv_file: str):
    for name, similarity in filtered_list:
        if name == csv_file:
            return similarity
    return 0

for file in os.listdir("Log_results"):
    try:
        os.rename("Log_results/" + file, f"Log_results/similarity_{get_similarity(file)}_" + file)
    except FileNotFoundError:
        print("file not found")
        pass


NameError: name 'similarity_dict' is not defined